In [1]:
import os
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from sklearn.model_selection import train_test_split

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import keras
from keras.applications.resnet50 import ResNet50
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout, Activation
from keras.models import Sequential
from keras.utils import to_categorical

In [3]:
data = pd.read_csv('./pokemon_aug.csv')
data = data.drop(['ID','Total', 'HP', 'Attack', 'Defense','Sp. Atk','Sp. Def',
                  'Speed', 'Generation', 'Legendary'], axis='columns')

In [ ]:
data.Type1.unique()

In [4]:
bug = os.listdir('images/scraped/bug/resize/')
grass = os.listdir('images/scraped/grass/resize/')
fire = os.listdir('images/scraped/fire/resize/')
water = os.listdir('images/scraped/water/resize/')
normal = os.listdir('images/scraped/normal/resize/')
poison = os.listdir('images/scraped/poison/resize/')
ele = os.listdir('images/scraped/ele/resize/')
ground = os.listdir('images/scraped/ground/resize/')
fairy = os.listdir('images/scraped/fairy/resize/')
fight = os.listdir('images/scraped/fight/resize/')
psy = os.listdir('images/scraped/psy/resize/')
rock = os.listdir('images/scraped/rock/resize/')
ghost = os.listdir('images/scraped/ghost/resize/')
ice = os.listdir('images/scraped/ice/resize/')
drag = os.listdir('images/scraped/dragon/resize/')
dark = os.listdir('images/scraped/dark/resize/')
steel = os.listdir('images/scraped/steel/resize/')
fly = os.listdir('images/scraped/fly/resize/')

In [5]:
bug = pd.DataFrame(bug, columns=['fileid'])
grass = pd.DataFrame(grass, columns=['fileid'])
fire = pd.DataFrame(fire, columns=['fileid'])
water = pd.DataFrame(water, columns=['fileid'])
normal = pd.DataFrame(normal, columns=['fileid'])
poison = pd.DataFrame(poison, columns=['fileid'])
ele = pd.DataFrame(ele, columns=['fileid'])
ground = pd.DataFrame(ground, columns=['fileid'])
fairy = pd.DataFrame(fairy, columns=['fileid'])
fight = pd.DataFrame(fight, columns=['fileid'])
psy = pd.DataFrame(psy, columns=['fileid'])
rock = pd.DataFrame(rock, columns=['fileid'])
ghost = pd.DataFrame(ghost, columns=['fileid'])
ice = pd.DataFrame(ice, columns=['fileid'])
drag = pd.DataFrame(drag, columns=['fileid'])
dark = pd.DataFrame(dark, columns=['fileid'])
steel = pd.DataFrame(steel, columns=['fileid'])
fly = pd.DataFrame(fly, columns=['fileid'])

In [6]:
bug['Type1'] = "Bug"
grass['Type1'] = "Grass"
fire['Type1'] = "Fire"
water['Type1'] = "Water"
normal['Type1'] = "Normal"
poison['Type1'] = "Poison"
ele['Type1'] = "Electric"
ground['Type1'] = "Ground"
fairy['Type1'] = "Fairy"
fight['Type1'] = "Fight"
psy['Type1'] = "Psychic"
rock['Type1'] = "Rock"
ghost['Type1'] = "Ghost"
ice['Type1'] = "Ice"
drag['Type1'] = "Dragon"
dark['Type1'] = "Dark"
steel['Type1'] = "Steel"
fly['Type1'] = "Flying"

In [7]:
data.append([bug, grass, fire, water, normal, poison, ele, ground, fairy, fight, psy, rock, ghost, ice, drag, dark, steel, fly], sort=True)

,Name,Type1,Type2,fileid
0,Bulbasaur,Grass,Poison,1.png
1,Ivysaur,Grass,Poison,2.png
2,Venusaur,Grass,Poison,3.png
3,VenusaurMega Venusaur,Grass,Poison,3-mega.png
4,Charmander,Fire,NaN,4.png
5,Charmeleon,Fire,NaN,5.png
6,Charizard,Fire,Flying,6.png
7,CharizardMega Charizard X,Fire,Dragon,6-mega-x.png
8,CharizardMega Charizard Y,Fire,Flying,6-mega-y.png
9,Squirtle,Water,NaN,7.png


In [39]:
data = data.drop(columns=['Name', 'Type2'])
data.to_csv('pokemon_aug_plus_scraped.csv')

In [40]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 793 entries, 0 to 792
Data columns (total 2 columns):
Type1     793 non-null object
fileid    793 non-null object
dtypes: object(2)
memory usage: 12.5+ KB


In [27]:
len(data)

793

----

In [28]:
mask = np.random.rand(len(data)) < 0.8

In [29]:
train = data[mask].reset_index()
test = data[~mask].reset_index()

In [30]:
len(train)

660

In [31]:
len(test)

133

In [32]:
gen = ImageDataGenerator(vertical_flip=True, validation_split=0.2)

In [33]:
traingen = gen.flow_from_dataframe(train, directory='./images/comp/', x_col='fileid', y_col='Type1',
                                   class_mode='categorical', subset='training', batch_size=5)
validgen = gen.flow_from_dataframe(train, directory='./images/comp/', x_col='fileid', y_col='Type1',
                                   class_mode='categorical', subset='validation', batch_size=5)

Found 174 images belonging to 18 classes.
Found 156 images belonging to 18 classes.


In [34]:
testdatagen = ImageDataGenerator(vertical_flip=False)
testgen = testdatagen.flow_from_dataframe(test, directory='./images/comp/', x_col='fileid', y_col='Type1', batch_size=5)

Found 75 images belonging to 18 classes.


In [35]:
model = Sequential()
model.add(Conv2D(32, kernel_size=2, padding='same', activation='relu', 
                 input_shape=(256, 256, 3)))
model.add(Conv2D(32, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, kernel_size=2, activation='relu'))
model.add(Conv2D(64, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())

model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(18, activation='softmax'))

In [36]:
model.compile(optimizer='adam', loss='categorical_crossentropy')

In [37]:
STEP_SIZE_TRAIN=traingen.n//traingen.batch_size
STEP_SIZE_VALID=validgen.n//validgen.batch_size
step_size_test=testgen.n//testgen.batch_size

In [38]:
model.fit_generator(generator=traingen,
                    epochs=10,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                   validation_data=validgen,
                   validation_steps=STEP_SIZE_VALID)

Epoch 1/10
34/34 [==============================] - 92s 3s/step - loss: 15.5284 - val_loss: 15.3902
Epoch 2/10
 9/34 [======>.......................] - ETA: 55s - loss: 15.3122

KeyboardInterrupt: 

In [ ]:
testgen.reset()
pred = model.predict_generator(testgen, steps = step_size_test, verbose=1)

In [ ]:
pred[2]

In [ ]:
predicted_class_indices=np.argmax(pred,axis=1)

In [ ]:
predicted_class_indices

In [ ]:
labels = (testgen.class_indices)
print(labels)

In [ ]:
labels = dict((v,k) for k,v in labels.items())
print(labels)

In [ ]:
predictions = [labels[k] for k in predicted_class_indices]
print(predictions[-1])

In [ ]:
len(predictions)

In [ ]:
len(testgen.filenames)

In [ ]:
filenames=testgen.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
results.to_csv("results.csv",index=False)

In [ ]:
results.groupby

In [ ]:
data.head()

In [ ]:
data.info()